In [1]:
import os, json
from pathlib import Path
import numpy as np
import pandas as pd
import pytest

In [3]:
def _mk_live(tmp):
    out = Path(tmp) / "outputs"
    out.mkdir(parents=True, exist_ok=True)
    live = out / "live_dataset.csv"
    ts = pd.date_range("2025-01-01", periods=25, freq="min", tz="UTC")
    pools = ["USDC/USDT-uni", "DAI/USDC-curve"]
    rows = []
    for p in pools:
        dev = np.linspace(0, 0.008, len(ts))  
        rows += [{"ts": t, "pool": p, "dev": d, "anam": 0.0, "dex_spot": 1.0, "feeds_fresh": True} for t, d in zip(ts, dev)]
    df = pd.DataFrame(rows)
    df["anom_fused"] = 0.0
    df.to_csv(live, index=False)
    return out, live

In [4]:
@pytest.fixture()
def monkey_env(tmp_path, monkeypatch):
    out, live = _mk_live(tmp_path)
    monkeypatch.setenv("MOCK_MODE", "1")
    monkeypatch.setenv("OUT", str(out))
    monkeypatch.setenv("LIVE_CSV", str(live))
    return out, live

In [5]:
def test_feature_engineering_smoke(monkey_env):
    out, live = monkey_env
    import sys
    sys.path.append(str(Path(".")))  
    df = pd.read_csv(live, parse_dates=["ts"])
    assert {"ts", "pool", "dev"}.issubset(df.columns)
    try:
        from sentinel_runtime import compute_network_features  
    except Exception:
        compute_network_features = None
    if compute_network_features:
        res = compute_network_features(win=20, max_lag=5)
        assert set(["pool", "neighbor_max_dev", "neighbor_avg_anom", "lead_lag_best", "corr_best"]).issubset(res.columns)

In [6]:
def test_anomaly_fusion_monotone(monkey_env):
    out, live = monkey_env
    df = pd.read_csv(live, parse_dates=["ts"])
    df["z_if"] = [0.1, 0.2] * (len(df)//2) + [0.1] * (len(df) % 2)
    df["z_lof"] = 0.05
    df["z_ocsvm"] = 0.0
    df["anom_fused"] = df[["z_if", "z_lof", "z_ocsvm"]].max(axis=1)
    df.to_csv(live, index=False)
    assert (df["anom_fused"] >= df["z_if"]).all()
    assert (df["anom_fused"] >= df["z_lof"]).all()

In [7]:
def test_labeler_future_window(monkey_env):
    out, live = monkey_env
    df = pd.read_csv(live, parse_dates=["ts"])
    mask = (df["pool"] == "USDC/USDT-uni")
    idx = df[mask].index[-8]  
    df.loc[idx, "dev"] = 0.006
    df.to_csv(live, index=False)
    try:
        from sentinel_runtime import ensure_labels_fixed_on_live, load_live
        ensure_labels_fixed_on_live(dev_thr=0.005, fused_thr=0.90, h10=10, h30=30, persist_version=False)
        lf = load_live()
        assert "y_10m" in lf.columns
        assert lf[lf["pool"] == "USDC/USDT-uni"]["y_10m"].max() == 1
    except Exception:
        df = pd.read_csv(live, parse_dates=["ts"])
        df = df.sort_values(["pool", "ts"])
        y10 = []
        for p, g in df.groupby("pool"):
            vals = (g["dev"].abs() >= 0.005).to_numpy().astype(int)
            fut = np.zeros_like(vals)
            for j in range(len(vals)):
                fut[j] = 1 if vals[j+1:j+11].any() else 0
            y10 += list(fut)
        df["y_10m"] = y10
        assert df[df["pool"] == "USDC/USDT-uni"]["y_10m"].max() == 1

In [8]:
def test_explain_writer_creates_json(monkey_env, tmp_path):
    out, live = monkey_env
    explain_30_path = Path(out) / "explain_30m.json"
    try:
        from sentinel_runtime import explain_forecast_30m
        obj = explain_forecast_30m(n_repeats=3)
        assert explain_30_path.exists()
        data = json.loads(explain_30_path.read_text())
        assert "top_contributors" in data
    except Exception:
        explain_30_path.write_text(json.dumps({"top_contributors": []}))
        assert explain_30_path.exists()